In [4]:
import matplotlib.pylab as pl
import matplotlib.pyplot as plt

import gnn
from rdkit import Chem
from rdkit.Chem import Draw
import utils
import util_metrics

import torch
import torch.nn as nn
import torch.nn.functional as F

import gnn_decoder
from torch_geometric.data import Data,Batch
from torch_geometric.utils import to_dense_adj,to_dense_batch,get_embeddings
from torch.utils.data import random_split
from torch_geometric.loader import DataLoader
import vgae
import copy
import visualize
import wandb
import utils
import matching
from decoder import Decoder
import torch.nn.functional as F
import numpy as np
import gzip
import pickle
from tqdm import tqdm

In [2]:

file_path='../data/QM9_canonical.pkl'
with open(file_path, 'rb') as file:
    dataset_smiles = pickle.load(file)

In [5]:
with gzip.open('../data/dataset_small.pkl.gz', 'rb') as f:
    dataset = pickle.load(f)

In [6]:
latent_dims=100
edge_dim=4
node_dim=4
train_loader=DataLoader(dataset,4,shuffle=True)
matcher=matching.NaiveMatcher()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder=gnn.GINE(in_channels=node_dim,hidden_channels=[16,32,64,128],out_channels=128,norm="batch",num_layers=4,
                  act="leakyrelu",edge_dim=4,dropout=0.1).to(device)
#decoder=gnn_decoder.Generator_Decoder(latent_dim=latent_dims,node_dim=node_dim,edge_dim=edge_dim,num_layers=4).to(device)
decoder=Decoder(latent_dims)
matcher=matching.NaiveMatcher()

vgae_model=vgae.VGAE(encoder=encoder,decoder=decoder,latent_dims=latent_dims,embedding="graph",eval=False).to(device)
#vgae_model.set_encoder()

In [7]:
vgae_model.train()
optimizer = torch.optim.Adam(vgae_model.parameters(), lr=1e-5,weight_decay=1e-5)
kl_loss_ls,matched_loss_ls,unmatched_loss_ls,num_loss_ls=[],[],[],[]
alpha=0.1
beta=0.1

for epoch in tqdm(range(2)):
    kl_ls=[]
    matched_ls=[]
    unmatched_ls=[]
    num_ls=[]
    #vgae_model.set_encoder()
    for data in train_loader:
      data = data.to(device)
      optimizer.zero_grad()
      data_rec,mu,logvar,_,pred_num= vgae_model(data)

      num_nodes=torch.bincount(data.batch).to(torch.float32)
      num_loss=alpha*F.mse_loss(pred_num,num_nodes)

      matched_loss,unmatched_loss=matcher.diffusion_matching(data,data_rec,test=False)
      
     
      kl_loss=beta*vgae_model.kl_loss(mu,logvar)
      loss=matched_loss+kl_loss+num_loss
      loss.backward()
      optimizer.step()
      #Log data and evaluate reconstruction quality
      #valid_rec,num_rec=util_metrics.evaluate_reconstruction(data,data_rec)

      optimizer.step()
      kl_ls.append(kl_loss.item())
      matched_ls.append(matched_loss.item())
      unmatched_ls.append(unmatched_loss.item())
      num_ls.append(num_loss.item())
    kl_loss_ls.append(np.array(kl_ls).mean())
    matched_loss_ls.append(np.array(matched_ls).mean())
    unmatched_loss_ls.append(np.array(unmatched_ls).mean())
    num_loss_ls.append(np.array(num_ls).mean())
      




  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [01:28<?, ?it/s]


KeyboardInterrupt: 

In [6]:
torch.cat([torch.tensor(1).unsqueeze(0),torch.tensor([2])])

tensor([1, 2])

In [7]:
from torch_geometric.nn.models import GIN,PNA
from torch_geometric.utils  import add_self_loops

PNA(in_channels=16+9,hidden_channels=32,num_layers=3,out_channels=32,dropout=0.,act="leakyrelu",aggregators=["sum","mean","max"],scalers="linear",
    deg=)

SyntaxError: expected argument value expression (1132239008.py, line 3)

In [13]:
num_nodes = max(torch.max(data.edge_index) for data in dataset) + 1
total_degrees = torch.zeros(num_nodes, dtype=torch.int32)
for data in dataset:
    degrees = torch.zeros(num_nodes, dtype=torch.int32)
    for edge in zip(data.edge_index[0],data.edge_index[1]):
        u, v = edge
        degrees[u] += 1
        degrees[v] += 1  # Undirected graph means both nodes share the edge
    total_degrees += degrees

In [14]:
from torch_geometric.utils import  degree
DEVICE="cpu"
def dataset_to_degree_bin(train_dataset):
  """
  Convert a dataset to a histogram of node degrees (in-degrees).
  Load from file if available; otherwise, compute from the dataset.
  """



  # Compute the maximum in-degree in the training data.
  max_degree = -1
  for data in train_dataset:
    data = data
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    max_degree = max(max_degree, int(d.max()))

  # Create an empty histogram for degrees.
  deg = torch.zeros(max_degree + 1, dtype=torch.long, device=DEVICE)

  # Populate the histogram with data from the dataset.
  for data in train_dataset:
    data = data.to(DEVICE)
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())


  return deg

In [15]:
dataset_to_degree_bin(dataset)

tensor([   0, 4223, 7698, 4767,  927])

In [5]:
torch.rand((8,6)).unsqueeze(-1)

tensor([[[0.4464],
         [0.8829],
         [0.1657],
         [0.8372],
         [0.3358],
         [0.5789]],

        [[0.3690],
         [0.9846],
         [0.9116],
         [0.2970],
         [0.2022],
         [0.7362]],

        [[0.1399],
         [0.6301],
         [0.7738],
         [0.6097],
         [0.9382],
         [0.8478]],

        [[0.3273],
         [0.4601],
         [0.5417],
         [0.1121],
         [0.2040],
         [0.1708]],

        [[0.9455],
         [0.4811],
         [0.9671],
         [0.3204],
         [0.8872],
         [0.4423]],

        [[0.1172],
         [0.7680],
         [0.9519],
         [0.3635],
         [0.9462],
         [0.6773]],

        [[0.5267],
         [0.0653],
         [0.1442],
         [0.7328],
         [0.4561],
         [0.6171]],

        [[0.2665],
         [0.8870],
         [0.4843],
         [0.1504],
         [0.6513],
         [0.0931]]])

In [9]:
import torch

# Example tensor of shape (n, 9, 9, 4)
n = 3  # Example batch size
tensor = torch.rand((9, 9, 4))

# Create a new tensor with an additional 0 at the start of the fourth dimension
zeros = torch.zeros(( 9, 9, 1))
expanded_tensor = torch.cat((zeros, tensor), dim=2)

print(expanded_tensor.shape)  # Should be (n, 9, 9, 5)


torch.Size([9, 9, 5])


Evaluate Model

In [ ]:
vgae_model.eval()
sample=vgae_model.sample(wandb.config["sample_size"])    
validity,unique,novelty,_=util_metrics.compute_metrics(sample,dataset_smiles)

In [ ]:
PATH="YOUR/PATH/HERE"
torch.save(vgae_model.state_dict(), PATH)

Generate Samples. Compute fraction of valid, unique, and novel molecules. Return SMILES representation of all novel SMILES.
Plot generated molecules

In [ ]:
vgae_model.eval()
sample=vgae_model.sample(1000)
validity,unique,novel,smiles_list=util_metrics.compute_metrics(sample,dataset_smiles)

In [ ]:
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import AllChem as Chem
mol_list = [Chem.MolFromSmiles(smiles) for smiles in list(smiles_list)]
Draw.MolsToGridImage(mol_list[:50], molsPerRow=10, subImgSize=(400,400),maxMols=100)